In [1]:
import os
from dotenv import load_dotenv
import psycopg2
import requests
from typing import Dict, List
import logging 

In [2]:
load_dotenv('/home/wjones/CC/Capstone/tbd2/LACCTiC/.env', override=True)


True

In [4]:
logging.basicConfig(filename='db_query.log', filemode='w', format='%(name)s - %(levelname)s - %(message)s', level=logging.INFO)


In [5]:
db_name = os.getenv('DB_NAME')
db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASS')
db_host = os.getenv('DB_HOST')
db_port = os.getenv('DB_PORT')

In [6]:
# Connect to the database
db_params = {
    'dbname': db_name,
    'user': db_user,
    'password' : db_password,
    'host' : db_host,
    'port' : db_port
}

In [7]:
def connect_db(db_params):
    """Connect to the PostgreSQL database server."""
    conn = psycopg2.connect(**db_params)
    return conn

In [54]:
def get_personal_best(conn, first_name: str, last_name: str, distance: str) -> None:
    
    if distance == '8k':
        one = '8k'
        two = '8,000m'
        three = '8000m'
        four = '8.0k'
        five = "8000"
        six = "8"
    elif distance == '10k':
        one = '10k'
        two = '10,000m'
        three = '10000m'
        four = '10.0k'
        five = "10000"
        six = "10"
    elif distance == '5k':  
        one = '5k'
        two = '5,000m'
        three = '5000m'
        four = '5.0k'
        five = "5000"
        six = "5"
    elif distance == '6k':
        one = '6k'
        two = '6,000m'
        three = '6000m'
        four = '6.0k'
        five = "6000"
        six = "6"
    else:
        print("Invalid distance")
        return
    with conn.cursor() as cur:
        cur.execute("""
            SELECT 
                rn.firstname,
                rn.lastname,
                MIN(rr.time) AS personal_best_time
            FROM 
                Runners rn
            JOIN 
                RaceResults rr ON rn.lacctic_id = rr.lacctic_id
            JOIN 
                Races r ON rr.meet_id = r.meet_id
            WHERE 
                rn.firstname = %s AND rn.lastname = %s AND 
                (
                    r.section LIKE %s OR
                    r.section LIKE %s OR
                    r.section LIKE %s OR
                    r.section LIKE %s OR
                    r.section LIKE %s OR
                    r.section LIKE %s
                )
            GROUP BY 
                rn.firstname, rn.lastname;
        """, (first_name, last_name, "%"+one+"%", "%"+two+"%", "%"+three+"%", "%"+four+"%", "%"+five+"%", "%"+six+"%"))
        
        result = cur.fetchone()
        if result:
            print(f"Personal best for {result[0]} {result[1]}: {result[2]}")
        else:
            print(f"No data found for runner {first_name} {last_name}")


In [62]:
conn = connect_db(db_params)
print(get_personal_best(conn, 'Britt', 'Helgaas', '6k'))

Personal best for Britt Helgaas: 1407.2
None
